# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [71]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-18 01:18:54--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-18 01:18:54--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv.1’

cac_ltv_model.csv.1 100%[===================>] 645.90K  --.-KB

## Importar Datos

In [72]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



In [73]:
meta_ads = data[data["acquisition_channel"] == "meta_ads"]

resultado_meta = (
    meta_ads.groupby("acquisition_channel")
            .agg(n_clientes=("acquisition_channel", "size"),
                 ARPU_promedio=("arpu", "mean"))
            .round(2)
)

display(resultado_meta)

,n_clientes,ARPU_promedio
acquisition_channel,,
meta_ads,1771,168.79


El ARPU promedio es de 168,79 para los clientes de meta,

2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

In [74]:
latam_churn = data[(data["region"] == "LatAm") & (pd.to_numeric(data["churn_rate"], errors="coerce") > 0.05)]

resultado_latam = (
    latam_churn.groupby("region")
               .agg(conteo=("region", "size"),
                    ARPU_promedio=("arpu", "mean"))
               .round({"ARPU_promedio": 2})
)

display(resultado_latam)


,conteo,ARPU_promedio
region,,
LatAm,410,174.78


El conteo de clientes son de 410 y su ARPU promedio es de 174,78

3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

In [75]:
ent_long = data[
    (data["customer_tier"] == "Enterprise") &
    (pd.to_numeric(data["contract_length_months"], errors="coerce") > 6)
]

resultado_ent = (
    ent_long.groupby("customer_tier")
            .agg(gross_margin_promedio=("gross_margin", lambda s: pd.to_numeric(s, errors="coerce").mean()))
            .round(4)
)

display(resultado_ent)


,gross_margin_promedio
customer_tier,
Enterprise,0.82


El Gross Margin Promedio es del 82%

## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [76]:
resultado_b4 = (
    data.assign(arpu=pd.to_numeric(data["arpu"], errors="coerce"))
      .groupby("acquisition_channel", as_index=True)["arpu"]
      .mean()
      .sort_values(ascending=False)
      .to_frame(name="ARPU_promedio")
      .round(2)
)

display(resultado_b4)


,ARPU_promedio
acquisition_channel,
outbound_sales,173.61
google_ads,172.37
organic_search,171.14
meta_ads,168.79


El mayor es outbound_sales con 173,61 y el menor es meta_ads con 168,79

5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



In [77]:
resultado_b5 = (
    data.assign(churn_rate=pd.to_numeric(data["churn_rate"], errors="coerce"))
      .groupby("region", as_index=True)["churn_rate"]
      .mean()
      .sort_values(ascending=False)
      .to_frame(name="churn_promedio")
      .round(4)
)

display(resultado_b5)

if not resultado_b5.empty:
    region_top = resultado_b5.index[0]
    churn_top = resultado_b5.iloc[0, 0]
    print(f"Región con mayor churn: {region_top} ({churn_top:.4f})")
else:
    print("No hay datos para calcular churn por región.")


,churn_promedio
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


Región con mayor churn: Middle East (0.0539)


La Tabla muestra que el comun es 0,05 pero la reguin con mayor churm es
#Middle East (0,0539)

6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

In [78]:
resultado_b6 = (
    data.assign(marketing_spend=pd.to_numeric(data["marketing_spend"], errors="coerce"))
      .groupby("customer_tier", as_index=True)["marketing_spend"]
      .sum()
      .sort_values(ascending=False)
      .to_frame(name="marketing_spend_total")
      .round(2)
)

display(resultado_b6)


if not resultado_b6.empty:
    top_tier = resultado_b6.index[0]
    top_spend = resultado_b6.iloc[0, 0]
    print(f"Tier con mayor gasto de marketing: {top_tier} ({top_spend:,.2f})")
else:
    print("No hay datos para calcular marketing_spend por tier.")


,marketing_spend_total
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"


Tier con mayor gasto de marketing: Pro (589,265.04)


El tier que consume mas presupuesto es
#Pro con 589,265.04

## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [79]:
web = data[data["signup_source"] == "web"].copy()

resultado_c7 = (
    web.assign(churn_rate=pd.to_numeric(web["churn_rate"], errors="coerce"))
       .groupby("acquisition_channel", as_index=True)["churn_rate"]
       .mean()
       .sort_values(ascending=True)   # menor churn = mejor
       .to_frame(name="churn_promedio_web")
       .round(4)
)

display(resultado_c7)

if not resultado_c7.empty:
    canal_mejor = resultado_c7.index[0]
    churn_mejor = resultado_c7.iloc[0, 0]
    print(f"Mejor (menor churn) en web: {canal_mejor} ({churn_mejor:.4f})")
else:
    print("No hay registros con signup_source = 'web'.")


,churn_promedio_web
acquisition_channel,
outbound_sales,0.05
organic_search,0.05
meta_ads,0.05
google_ads,0.05


Mejor (menor churn) en web: outbound_sales (0.0516)


Los churn por canal en web son prácticamente iguales (~0.05); la diferencia de 0.0516 vs 0.050x es mínima y se “pierde” al redondear a 2 decimales.

Con empate técnico en churn, prioriza el canal con CAC más bajo y/o LTV más alto en web para asignar presupuesto.

8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [80]:
mob = data[data["signup_source"] == "mobile_app"].copy()

resultado_c8 = (
    mob.assign(arpu=pd.to_numeric(mob["arpu"], errors="coerce"))
       .groupby("region", as_index=True)["arpu"]
       .mean()
       .sort_values(ascending=False)
       .to_frame(name="ARPU_promedio_mobile")
       .round(2)
)

display(resultado_c8)

if not resultado_c8.empty:
    region_top = resultado_c8.index[0]
    arpu_top = resultado_c8.iloc[0, 0]
    print(f"Región con mayor ARPU (mobile_app): {region_top} ({arpu_top:.2f})")
else:
    print("No hay registros con signup_source = 'mobile_app'.")


,ARPU_promedio_mobile
region,
North America,174.00
LatAm,173.20
Africa,172.60
Middle East,171.71
APAC,169.09
Europe,165.89


Región con mayor ARPU (mobile_app): North America (174.00)


La Región con mayor ARPU (mobile_app) es
#North America (174.00)

9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [81]:
pro = data[data["customer_tier"] == "Pro"].copy()

resultado_c9 = (
    pro.assign(marketing_spend=pd.to_numeric(pro["marketing_spend"], errors="coerce"))
       .groupby("acquisition_channel", as_index=True)["marketing_spend"]
       .sum()
       .sort_values(ascending=False)
       .to_frame(name="marketing_spend_total_Pro")
       .round(2)
)

display(resultado_c9)

if not resultado_c9.empty:
    canal_top = resultado_c9.index[0]
    gasto_top = resultado_c9.iloc[0, 0]
    print(f"Canal con mayor gasto (Pro): {canal_top} ({gasto_top:,.2f})")
else:
    print("No hay registros de clientes Pro.")


,marketing_spend_total_Pro
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
outbound_sales,"121,177.04"
organic_search,"29,848.72"


Canal con mayor gasto (Pro): google_ads (226,721.68)


En el marketing_spend_total_Pro google_ads es el que más presupuesto consumió en el tier Pro (top 1)

## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [82]:
import numpy as np
tmp = (
    data.assign(marketing_spend=pd.to_numeric(data["marketing_spend"], errors="coerce"))
      .groupby("acquisition_channel", as_index=True)
      .agg(total_spend=("marketing_spend","sum"),
           n_clientes=("acquisition_channel","size"))
)

cac_por_canal = (
    tmp.assign(CAC = tmp["total_spend"] / tmp["n_clientes"].replace(0, np.nan))
       .loc[:, ["CAC", "total_spend", "n_clientes"]]
       .sort_values("CAC", ascending=True)
       .round({"CAC": 2, "total_spend": 2})
)

display(cac_por_canal)


,CAC,total_spend,n_clientes
acquisition_channel,,,
organic_search,49.95,"88,917.87",1780
outbound_sales,200.22,"354,391.70",1770
meta_ads,349.27,"618,560.13",1771
google_ads,401.02,"696,163.32",1736


El canal más eficiente es organic_search (CAC 49.95, n=1780). El menos eficiente es google_ads (CAC 401.02, n=1736).

11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [83]:
tmp_tier = (
    data.assign(marketing_spend=pd.to_numeric(data["marketing_spend"], errors="coerce"))
      .groupby("customer_tier", as_index=True)
      .agg(total_spend=("marketing_spend","sum"),
           n_clientes=("customer_tier","size"))
)

cac_por_tier = (
    tmp_tier.assign(CAC = tmp_tier["total_spend"] / tmp_tier["n_clientes"].replace(0, np.nan))
            .loc[:, ["CAC", "total_spend", "n_clientes"]]
            .sort_values("CAC", ascending=True)
            .round({"CAC": 2, "total_spend": 2})
)

display(cac_por_tier)


,CAC,total_spend,n_clientes
customer_tier,,,
Pro,248.43,"589,265.04",2372
Enterprise,249.19,"584,348.87",2345
Basic,249.75,"584,419.11",2340


El CAC más bajo es Pro = 248.43, seguido muy de cerca por Enterprise = 249.19 y Basic = 249.75.

12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [84]:
data = data.assign(
    arpu=pd.to_numeric(data["arpu"], errors="coerce"),
    contract_length_months=pd.to_numeric(data["contract_length_months"], errors="coerce"),
    gross_margin=pd.to_numeric(data["gross_margin"], errors="coerce"),
    churn_rate=pd.to_numeric(data["churn_rate"], errors="coerce"),
)

data["ltv_individual"] = (
    data["arpu"] * data["contract_length_months"] * data["gross_margin"] * (1 - data["churn_rate"])
)

valid = data["ltv_individual"].notna()
data_ltv = data[valid]




In [85]:
ltv_por_tier = (
    data_ltv.groupby("customer_tier", as_index=True)["ltv_individual"]
            .mean()
            .sort_values(ascending=False)
            .to_frame(name="LTV_promedio")
            .round(2)
)

display(ltv_por_tier)

if not ltv_por_tier.empty:
    tier_top = ltv_por_tier.index[0]
    ltv_top = ltv_por_tier.iloc[0, 0]
    print(f"Tier con mayor LTV promedio: {tier_top} ({ltv_top:.2f})")

,LTV_promedio
customer_tier,
Enterprise,"1,349.02"
Pro,577.63
Basic,291.77


Tier con mayor LTV promedio: Enterprise (1349.02)


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [86]:
ltv_por_canal = (
    data.assign(ltv_individual=pd.to_numeric(data["ltv_individual"], errors="coerce"))
        .groupby("acquisition_channel", as_index=True)["ltv_individual"]
        .mean()
        .sort_values(ascending=False)
        .to_frame(name="LTV_promedio")
        .round(2)
)

display(ltv_por_canal)

,LTV_promedio
acquisition_channel,
organic_search,765.13
outbound_sales,751.57
meta_ads,729.05
google_ads,710.25


LTV promedio por canal

LTV: organic_search 765.13 > outbound_sales 751.57 > meta_ads 729.05 > google_ads 710.25.

Con tus CAC (49.95, 200.22, 349.27, 401.02), los ratios LTV/CAC aproximados son:

organic_search ≈ 15.3× (muy eficiente)

outbound_sales ≈ 3.8× (aceptable)

meta_ads ≈ 2.1× (insuficiente)

google_ads ≈ 1.8× (insuficiente)
Se debe priorizar orgánico; mantener outbound donde el mix de tiers sea favorable; optimizar o recortar Meta/Google salvo segmentos muy rentables.

12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [87]:
ltv_por_tier = (
    data.assign(ltv_individual=pd.to_numeric(data["ltv_individual"], errors="coerce"))
        .groupby("customer_tier", as_index=True)["ltv_individual"]
        .mean()
        .sort_values(ascending=False)
        .to_frame(name="LTV_promedio")
        .round(2)
)

display(ltv_por_tier)


,LTV_promedio
customer_tier,
Enterprise,"1,349.02"
Pro,577.63
Basic,291.77


LTV promedio por tier

LTV: Enterprise 1,349.02 > Pro 577.63 > Basic 291.77.

Con CAC por tier ~$249, LTV/CAC aprox.: Enterprise ≈ 5.4× (rentable), Pro ≈ 2.3× (por debajo de 3×), Basic ≈ 1.17× (no rentable).
Se debe concentrar presupuesto en Enterprise; en Pro, elevar LTV o reducir CAC hasta ≥3×; en Basic, limitar inversión paga y usar canales de bajo costo.

13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [88]:
import numpy as np
import pandas as pd


In [89]:
data = data.assign(
    arpu=pd.to_numeric(data["arpu"], errors="coerce"),
    contract_length_months=pd.to_numeric(data["contract_length_months"], errors="coerce"),
    gross_margin=pd.to_numeric(data["gross_margin"], errors="coerce"),
    churn_rate=pd.to_numeric(data["churn_rate"], errors="coerce"),
    marketing_spend=pd.to_numeric(data["marketing_spend"], errors="coerce"),
)

if "ltv_individual" not in data.columns:
    data["ltv_individual"] = (
        data["arpu"]
        * data["contract_length_months"]
        * data["gross_margin"]
        * (1 - data["churn_rate"])
    )

data_sel = data


In [90]:
ltv_tier = (
    data_sel.groupby("customer_tier", as_index=True)["ltv_individual"]
            .mean()
)


In [91]:
cac_channel = (
    data_sel.groupby("acquisition_channel", as_index=True)
            .agg(total_spend=("marketing_spend","sum"),
                 n_clientes=("acquisition_channel","size"))
)
cac_channel["CAC"] = cac_channel["total_spend"] / cac_channel["n_clientes"].replace(0, np.nan)
cac_channel = cac_channel["CAC"]

In [92]:

ct = (
    data_sel.groupby(["acquisition_channel","customer_tier"], as_index=True)
            .agg(n_clientes=("customer_tier","size"))
)

ct = ct.copy()
ct["LTV_prom_tier"] = ct.index.get_level_values("customer_tier").map(ltv_tier)
ct["CAC_canal"]     = ct.index.get_level_values("acquisition_channel").map(cac_channel)


In [93]:
ct["LTV_CAC"] = ct["LTV_prom_tier"] / ct["CAC_canal"]
resultado_13 = (
    ct[["LTV_prom_tier","CAC_canal","LTV_CAC","n_clientes"]]
      .sort_values("LTV_CAC", ascending=False)
      .round({"LTV_prom_tier":2, "CAC_canal":2, "LTV_CAC":2})
)

display(resultado_13)

LTV_prom_tier  CAC_canal  LTV_CAC  \
acquisition_channel customer_tier                                      
organic_search      Enterprise          1,349.02      49.95    27.01   
                    Pro                   577.63      49.95    11.56   
outbound_sales      Enterprise          1,349.02     200.22     6.74   
organic_search      Basic                 291.77      49.95     5.84   
meta_ads            Enterprise          1,349.02     349.27     3.86   
google_ads          Enterprise          1,349.02     401.02     3.36   
outbound_sales      Pro                   577.63     200.22     2.88   
meta_ads            Pro                   577.63     349.27     1.65   
outbound_sales      Basic                 291.77     200.22     1.46   
google_ads          Pro                   577.63     401.02     1.44   
meta_ads            Basic                 291.77     349.27     0.84   
google_ads          Basic                 291.77     401.02     0.73   

                                   n_clientes  
acquisition_channel customer_tier              
organic_search      Enterprise            592  
                    Pro                   597  
outbound_sales      Enterprise            589  
organic_search      Basic                 591  
meta_ads            Enterprise            564  
google_ads          Enterprise            600  
outbound_sales      Pro                   605  
meta_ads            Pro                   605  
outbound_sales      Basic                 576  
google_ads          Pro                   565  
meta_ads            Basic                 602  
google_ads          Basic                 571

In [94]:
saludables = resultado_13[resultado_13["LTV_CAC"] > 3]
print("\nCombinaciones saludables (LTV/CAC > 3):")
if not saludables.empty:
    display(saludables)
else:
    print("No hay combinaciones con LTV/CAC > 3 en esta selección.")




Combinaciones saludables (LTV/CAC > 3):


LTV_prom_tier  CAC_canal  LTV_CAC  \
acquisition_channel customer_tier                                      
organic_search      Enterprise          1,349.02      49.95    27.01   
                    Pro                   577.63      49.95    11.56   
outbound_sales      Enterprise          1,349.02     200.22     6.74   
organic_search      Basic                 291.77      49.95     5.84   
meta_ads            Enterprise          1,349.02     349.27     3.86   
google_ads          Enterprise          1,349.02     401.02     3.36   

                                   n_clientes  
acquisition_channel customer_tier              
organic_search      Enterprise            592  
                    Pro                   597  
outbound_sales      Enterprise            589  
organic_search      Basic                 591  
meta_ads            Enterprise            564  
google_ads          Enterprise            600

 la combinación más saludable es ('organic_search', 'Enterprise') con LTV_prom_tier=1349.02, CAC_canal=49.95 y LTV/CAC=27.01.

## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.


# Parte E

## 13) CMO por un día: ¿en qué canal(es) invertir el próximo trimestre y por qué?
**Decisión:** priorizar **organic_search** y, en segundo lugar, **outbound_sales**.

**Citas clave (de los resultados):**
- **CAC por canal**: organic_search = **49.95**; outbound_sales = **200.22**; meta_ads = **349.27**; google_ads = **401.02**.
- **Churn por canal (web)**: ~**0.05** sin diferencias materiales al redondear.

**Razonamiento breve:**
- **organic_search** combina **CAC muy bajo** y buen volumen, maximizando ROI de adquisición.
- **outbound_sales** es defendible cuando la mezcla trae **Enterprise** (LTV alto) y mantiene **LTV/CAC > 3**.
- **meta_ads** y **google_ads** sólo se justifican en segmentos con **LTV/CAC ≥ 3**; de lo contrario, optimizar o recortar.

---

## 14) Región con mayor riesgo e hipótesis de churn
**Hallazgo:** el **churn promedio por región = 0.05** en todas las regiones observadas. No hay una región “peor” en el agregado.

**Hipótesis a validar con cortes más finos (región × canal / signup_source):**
- **Onboarding/soporte:** barreras de idioma, huso horario, SLA.
- **Pagos:** métodos locales, tasas de rechazo.
- **Pricing/fit:** desalineación del plan con poder adquisitivo/madurez digital.
- **Tiempo a valor:** implementación lenta en Enterprise que eleva churn temprano.

---

## 15) Dos combinaciones canal + tier con mejor LTV/CAC y acciones
**Combinaciones saludables (LTV/CAC > 3):**
- **Enterprise + organic_search** ≈ **27×**
- **Enterprise + outbound_sales** ≈ **6.7×**
*(Referencias: Pro + organic_search ≈ 11.6×; Basic + organic_search ≈ 5.8×)*

**Acciones propuestas:**

**a) organic_search + Enterprise**  
- **Adquisición:** SEO de intención transaccional y mid/bottom-funnel (casos por industria, comparativas) + CRO en landing.  
- **Retención/valor:** onboarding acelerado con playbooks sectoriales y QBRs; asegurar adopción temprana para sostener churn ~0.05.

**b) outbound_sales + Enterprise**  
- **Adquisición:** calificación estricta de ICP (tamaño, industria), multithreading y secuencias por rol; priorizar cuentas con señales de intención (ABM/intent data).  
- **Retención/valor:** contratos anuales con hitos de adopción, health scores y CSM dedicado en los primeros 90 días.

---

### Apéndice (para defensa oral)
- **Canal:** organic_search es el más eficiente en CAC; outbound_sales funciona bien para Enterprise.
- **Tier:** Enterprise domina en LTV; Pro/Basic requieren elevar LTV o reducir CAC para superar 3×.
- **Presupuesto:** expandir **Enterprise** en **organic_search** y **outbound_sales**; mantener pago sólo donde **LTV/CAC ≥ 3**; optimizar o recortar el resto.